# Visuals Notebook for Resource Usage

In [ ]:
from helpers import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Load data for January 2024

In [ ]:
year_2024_01 = process_gpu_data('24', '01')
year_2024_01

In [ ]:
year_2024_01['scenario'].value_counts()

In [ ]:
node_status = pd.read_csv('/projectnb/scv/utilization/katia/queue_info.csv')
node_status_mapping = node_status.set_index('queuename')['class_own'].to_dict()
year_2024_01['class_own'] = year_2024_01['qname'].map(node_status_mapping)
node_status_mapping = node_status.set_index('queuename')['class_user'].to_dict()
year_2024_01['class_user'] = year_2024_01['qname'].map(node_status_mapping)

In [ ]:
# determine if job is ondemand/interactive vs batch
year_2024_01['job_interactive'] = (year_2024_01['job_name'].str.startswith("ood")) | (year_2024_01['job_name'] == "QRLOGIN")

In [ ]:
node_status

## Look at GPU average utilization over the time period

In [ ]:
# Convert 'time' to datetime
year_2024_01['time'] = pd.to_datetime(year_2024_01['time'], unit='s')

# Resample utilization by 1-hour intervals (mean utilization)
gpu_util_hourly = year_2024_01.resample('1H', on='time')['util'].mean()

# Fill nans with 0
gpu_util_hourly[gpu_util_hourly.isna()] = 0

# Resample utilization by 1-hour intervals (mean utilization)
gpu_util_hourly_buyin = year_2024_01[year_2024_01['class_own']=='buyin'].resample('1H', on='time')['util'].mean()
gpu_util_hourly_shared = year_2024_01[year_2024_01['class_own']=='shared'].resample('1H', on='time')['util'].mean()

# Fill nans with 0
gpu_util_hourly[gpu_util_hourly.isna()] = 0
gpu_util_hourly_buyin[gpu_util_hourly_buyin.isna()] = 0
gpu_util_hourly_shared[gpu_util_hourly_shared.isna()] = 0



In [ ]:
# Define moving average window size
window_size = 10  # Adjust as needed

# Compute moving averages
gpu_util_hourly_smooth = gpu_util_hourly.rolling(window=window_size, min_periods=1).mean()
gpu_util_hourly_buyin_smooth = gpu_util_hourly_buyin.rolling(window=window_size, min_periods=1).mean()
gpu_util_hourly_shared_smooth = gpu_util_hourly_shared.rolling(window=window_size, min_periods=1).mean()

# Create subplots: 2 rows, 1 column
fig, axes = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plot Overall Usage on the first subplot
sns.lineplot(ax=axes[0], x=gpu_util_hourly.index, y=gpu_util_hourly_smooth.values, color="gray", label="Overall Usage (Smoothed)")
axes[0].axhline(gpu_util_hourly_smooth.mean(), color="gray", linestyle="dashed", linewidth=1, label="Overall Avg")
axes[0].set_ylabel("GPU Utilization (%)")
axes[0].set_title("Overall Hourly GPU Utilization (Smoothed)")
axes[0].legend()
axes[0].grid()

# Plot Buyin & Shared Usage on the second subplot
sns.lineplot(ax=axes[1], x=gpu_util_hourly_buyin.index, y=gpu_util_hourly_buyin_smooth.values, color="orange", label="Buyin Usage (Smoothed)")
sns.lineplot(ax=axes[1], x=gpu_util_hourly_shared.index, y=gpu_util_hourly_shared_smooth.values, color="blue", label="Shared Usage (Smoothed)")
axes[1].axhline(gpu_util_hourly_buyin_smooth.mean(), color="orange", linestyle="dashed", linewidth=1, label="Buyin Avg")
axes[1].axhline(gpu_util_hourly_shared_smooth.mean(), color="blue", linestyle="dashed", linewidth=1, label="Shared Avg")
axes[1].set_xlabel("Time")
axes[1].set_ylabel("GPU Utilization (%)")
axes[1].set_title("Buyin & Shared Hourly GPU Utilization (Smoothed)")
axes[1].legend()
axes[1].grid()

# Adjust layout for better spacing
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

## Top GPU Users & Wasters by User, Project
### Summed by util, rather than count

In [ ]:
# Group by user and sum utilization
top_users = year_2024_01.groupby("user")["util"].sum().nlargest(10)

# Group by project and sum utilization
top_projects = year_2024_01.groupby("project_x")["util"].sum().nlargest(10)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# Create subplots for users and projects
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot top users
sns.barplot(x=top_users.values / 1200, y=top_users.index, ax=axes[0], palette="Blues_r")
axes[0].set_title("Top 10 Users by GPU Utilization", fontsize=14)
axes[0].set_xlabel("Total GPU Utilization (Hours)", fontsize=12)
axes[0].set_ylabel("User", fontsize=12)

# Plot top projects
sns.barplot(x=top_projects.values / 1200, y=top_projects.index, ax=axes[1], palette="Greens_r")
axes[1].set_title("Top 10 Projects by GPU Utilization", fontsize=14)
axes[1].set_xlabel("Total GPU Utilization (Hours)", fontsize=12)
axes[1].set_ylabel("Project", fontsize=12)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
year_2024_01[year_2024_01['user']=='vgiammar']

In [ ]:
# Convert GPU usage to GPU hours
year_2024_01["gpu_hours"] = year_2024_01["util"] * (1/12) * (1/100) #* year_2024_01["n_gpu"]
year_2024_01["wasted_gpu_hours"] = (100 - year_2024_01["util"]) * (1/12) * (1/100) #* year_2024_01["n_gpu"]

# Aggregate GPU hours and wasted GPU hours for users and projects
top_users_gpu = year_2024_01.groupby("user")[["gpu_hours", "wasted_gpu_hours"]].sum().nlargest(10, "gpu_hours")
top_projects_gpu = year_2024_01.groupby("project_x")[["gpu_hours", "wasted_gpu_hours"]].sum().nlargest(10, "gpu_hours")

# Get the top users and projects by wasted GPU hours (sorted properly)
top_users_wasted = top_users_gpu.sort_values("wasted_gpu_hours", ascending=True)
top_projects_wasted = top_projects_gpu.sort_values("wasted_gpu_hours", ascending=True)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# Create subplots for users and projects
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot top users (used & wasted)
top_users_gpu.sort_values("gpu_hours", ascending=True).plot(kind="barh", stacked=True, ax=axes[0, 0], color=["#1f77b4", "#ff7f0e"])
axes[0, 0].set_title("Top 10 Users by GPU Util Hours", fontsize=14)
axes[0, 0].set_xlabel("Total GPU Util Hours", fontsize=12)
axes[0, 0].set_ylabel("User", fontsize=12)
axes[0, 0].legend(["Used GPU Util Hours", "Wasted GPU Util Hours"])

# Plot top projects (used & wasted)
top_projects_gpu.sort_values("gpu_hours", ascending=True).plot(kind="barh", stacked=True, ax=axes[0, 1], color=["#2ca02c", "#d62728"])
axes[0, 1].set_title("Top 10 Projects by GPU Util Hours", fontsize=14)
axes[0, 1].set_xlabel("Total GPU Util Hours", fontsize=12)
axes[0, 1].set_ylabel("Project", fontsize=12)
axes[0, 1].legend(["Used GPU Util Hours", "Wasted GPU Util Hours"])

# Plot wasted GPU hours for users (Sorted by wasted_gpu_hours)
sns.barplot(x=top_users_wasted["wasted_gpu_hours"], y=top_users_wasted.index, ax=axes[1, 0], palette="Oranges_r")
axes[1, 0].set_title("Top 10 Users by Wasted GPU Util Hours", fontsize=14)
axes[1, 0].set_xlabel("Total Wasted GPU Util Hours", fontsize=12)
axes[1, 0].set_ylabel("User", fontsize=12)

# Plot wasted GPU hours for projects (Sorted by wasted_gpu_hours)
sns.barplot(x=top_projects_wasted["wasted_gpu_hours"], y=top_projects_wasted.index, ax=axes[1, 1], palette="Reds_r")
axes[1, 1].set_title("Top 10 Projects by Wasted GPU Util Hours", fontsize=14)
axes[1, 1].set_xlabel("Total Wasted GPU Util Hours", fontsize=12)
axes[1, 1].set_ylabel("Project", fontsize=12)

# Adjust layout
plt.tight_layout()
plt.show()

## Examine by Counts

In [ ]:
# Calculate the number of rows with zero utilization per user, note for scenario = 0
zero_util_users = year_2024_01[(year_2024_01['util'] == 0) & (year_2024_01['scenario'] != 0) & (year_2024_01['user'] != 'Missing Values')].dropna(subset=['user']).groupby('user').size().reset_index(name='zero_util_count')

# Calculate the number of rows with zero utilization per project
zero_util_projects = year_2024_01[(year_2024_01['util'] == 0) & (year_2024_01['scenario'] != 0) & (year_2024_01['user'] != 'Missing Values')].dropna(subset=['user']).groupby('project_x').size().reset_index(name='zero_util_count')

# Sort by zero_util_count in descending order for top users and projects
zero_util_users_sorted = zero_util_users.sort_values('zero_util_count', ascending=False).head(10)
zero_util_projects_sorted = zero_util_projects.sort_values('zero_util_count', ascending=False).head(10)

# Plot for top 10 users with zero GPU utilization
plt.figure(figsize=(10, 6))
sns.barplot(x='zero_util_count', y='user', data=zero_util_users_sorted, palette='Blues_d')
plt.title('Top 10 Users with Zero GPU Utilization')
plt.xlabel('Zero Utilization Count')
plt.ylabel('User')
plt.tight_layout()
plt.show()

# Plot for top 10 projects with zero GPU utilization
plt.figure(figsize=(10, 6))
sns.barplot(x='zero_util_count', y='project_x', data=zero_util_projects_sorted, palette='Blues_d')
plt.title('Top 10 Projects with Zero GPU Utilization')
plt.xlabel('Zero Utilization Count')
plt.ylabel('Project')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the total GPU minutes per user (all rows, regardless of utilization)
gpu_minutes_users = year_2024_01.groupby('user').size().reset_index(name='gpu_minutes_count')
gpu_minutes_users = gpu_minutes_users[(gpu_minutes_users['user'] != '-') & (gpu_minutes_users['user'] != 'Missing Values')]
# Calculate the total GPU minutes per project
gpu_minutes_projects = year_2024_01.groupby('project_x').size().reset_index(name='gpu_minutes_count')
gpu_minutes_projects = gpu_minutes_projects[(gpu_minutes_projects['project_x'] != '-') & (gpu_minutes_projects['project_x'] != 'Missing Values')]

# Convert GPU minutes to GPU hours (since each row represents 5 minutes, divide by 12 to get hours)
gpu_minutes_users['gpu_hours_count'] = gpu_minutes_users['gpu_minutes_count'] / 12
gpu_minutes_projects['gpu_hours_count'] = gpu_minutes_projects['gpu_minutes_count'] / 12

# Sort by gpu_hours_count in descending order for top users and projects
gpu_minutes_users_sorted = gpu_minutes_users.sort_values('gpu_hours_count', ascending=False).head(10)
gpu_minutes_projects_sorted = gpu_minutes_projects.sort_values('gpu_hours_count', ascending=False).head(10)

# Plot for top 10 users with the most GPU hours
plt.figure(figsize=(10, 6))
sns.barplot(x='gpu_hours_count', y='user', data=gpu_minutes_users_sorted, palette='viridis')
plt.title('Top 10 Users with Most GPU Hours')
plt.xlabel('GPU Hours')
plt.ylabel('User')
plt.tight_layout()
plt.show()

# Plot for top 10 projects with the most GPU hours
plt.figure(figsize=(10, 6))
sns.barplot(x='gpu_hours_count', y='project_x', data=gpu_minutes_projects_sorted, palette='viridis')
plt.title('Top 10 Projects with Most GPU Hours')
plt.xlabel('GPU Hours')
plt.ylabel('Project')
plt.tight_layout()
plt.show()

In [ ]:
# Aggregate wasted GPU counts by user and class_own
top_users_wasted_gpu = (
    year_2024_01[year_2024_01["user"]!='Missing Values'].groupby(["user", "class_own"])["wasted_gpu_hours"]
    .sum()
    .reset_index()
    .sort_values("wasted_gpu_hours", ascending=False)
    .head(20)  # Top 10 users by wasted GPU hours
)

# Set Seaborn style
sns.set_theme(style="whitegrid")

# Create the bar chart
plt.figure(figsize=(12, 6))
sns.barplot(
    x="wasted_gpu_hours", 
    y="user", 
    hue="class_own", 
    data=top_users_wasted_gpu, 
    palette="Reds_r"
)

# Customize the plot
plt.title("Top 10 Users with Most Wasted GPU Hours (Buyin vs Shared)", fontsize=14)
plt.xlabel("Total Wasted GPU Hours", fontsize=12)
plt.ylabel("User", fontsize=12)
plt.legend(title="Class Ownership")

# Show the plot
plt.show()


In [ ]:
# Filter and get top 10 users for each class_own (buyin & shared)
top_buyin_users = (
    year_2024_01[(year_2024_01["class_own"] == "buyin") & (year_2024_01["user"] != "Missing Values")]
    .groupby("user")["wasted_gpu_hours"]
    .sum()
    .reset_index()
    .sort_values("wasted_gpu_hours", ascending=False)
    .head(10)
)

top_shared_users = (
    year_2024_01[(year_2024_01["class_own"] == "shared") & (year_2024_01["user"] != "Missing Values")]
    .groupby("user")["wasted_gpu_hours"]
    .sum()
    .reset_index()
    .sort_values("wasted_gpu_hours", ascending=False)
    .head(10)
)

# Set up the figure with two bar charts
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart for Buyin Users
sns.barplot(
    x="wasted_gpu_hours", 
    y="user", 
    data=top_buyin_users, 
    palette="Blues_r", 
    ax=axes[0]
)
axes[0].set_title("Top 10 Buyin Users - Wasted GPU Hours")
axes[0].set_xlabel("Wasted GPU Hours")
axes[0].set_ylabel("User")

# Bar chart for Shared Users
sns.barplot(
    x="wasted_gpu_hours", 
    y="user", 
    data=top_shared_users, 
    palette="Reds_r", 
    ax=axes[1]
)
axes[1].set_title("Top 10 Shared Users - Wasted GPU Hours")
axes[1].set_xlabel("Wasted GPU Hours")
axes[1].set_ylabel("User")

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
top_shared_users

In [ ]:
top_shared_users_list = top_shared_users['user'].tolist()
top_shared_users_data = year_2024_01[year_2024_01["user"].isin(top_shared_users_list)]
top_shared_users_time_series = top_shared_users_data.groupby(["time", "user"])["util"].mean().reset_index()
top_shared_users_time_series["time"] = pd.to_datetime(top_shared_users_time_series["time"])

# Ensure the time column is in datetime format
top_shared_users_time_series["time"] = pd.to_datetime(top_shared_users_time_series["time"])

# Set the 'time' column as the index
top_shared_users_time_series.set_index("time", inplace=True)

# Resample to an hourly frequency, summing GPU hours for each hour
top_shared_users_time_series_hourly = top_shared_users_time_series.groupby('user').resample('H').mean().reset_index()

# Plotting the resampled data
plt.figure(figsize=(12, 6))
sns.lineplot(
    data=top_shared_users_time_series_hourly, 
    x="time", 
    y="util", 
    hue="user", 
    palette="tab10"
)
plt.title("Hourly GPU Util Over Time for Top Shared Users with Most Wasted GPU")
plt.xlabel("Time")
plt.ylabel("Util")
plt.xticks(rotation=45)
plt.legend(title="User", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True)
plt.show()



## Interactive/ood vs. Batch

In [ ]:
# Count the number of jobs for each type (ondemand/interactive vs batch)
job_type_count = year_2024_01['job_interactive'].value_counts().reset_index()
job_type_count.columns = ['Job Type', 'Count']
job_type_count['Job Type'] = job_type_count['Job Type'].map({True: 'Ondemand/Interactive', False: 'Batch'})

# Plot
plt.figure(figsize=(8, 6))
sns.barplot(x='Job Type', y='Count', data=job_type_count, palette="Set2")
plt.title('Job Type Count: Ondemand/Interactive vs Batch')
plt.xlabel('Job Type')
plt.ylabel('Number of Jobs')
plt.tight_layout()
plt.show()


In [ ]:
# Calculate total GPU usage for each job type
gpu_usage_by_job_type = year_2024_01.groupby('job_interactive')['gpu_hours'].sum().reset_index()
gpu_usage_by_job_type['job_interactive'] = gpu_usage_by_job_type['job_interactive'].map({True: 'Ondemand/Interactive', False: 'Batch'})

# Plot
plt.figure(figsize=(8, 6))
sns.barplot(x='job_interactive', y='gpu_hours', data=gpu_usage_by_job_type, palette="Set2")
plt.title('Total GPU Usage by Job Type')
plt.xlabel('Job Type')
plt.ylabel('Total GPU Hours')
plt.tight_layout()
plt.show()


In [ ]:
# Group by time and job type to get GPU usage over time
gpu_usage_time_by_type = year_2024_01.groupby([pd.Grouper(key='time', freq='1H'), 'job_interactive'])['util'].mean().reset_index()
gpu_usage_time_by_type['job_interactive'] = gpu_usage_time_by_type['job_interactive'].map({True: 'Ondemand/Interactive', False: 'Batch'})

# Plot
plt.figure(figsize=(12, 6))
sns.lineplot(data=gpu_usage_time_by_type, x='time', y='util', hue='job_interactive', palette="Set2")
plt.title('GPU Util Over Time by Job Type')
plt.xlabel('Date')
plt.ylabel('Avg GPU Util')
plt.xticks(rotation=45)
plt.legend(title="Job Type")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Filter for interactive jobs, excluding missing values
interactive_jobs = year_2024_01[
    year_2024_01["job_interactive"] & (year_2024_01["user"] != "Missing Values")
]

# Group by user and calculate sum of wasted GPU hours & average GPU utilization
interactive_wasted_gpu = (
    interactive_jobs.groupby("user")
    .agg(
        wasted_gpu_hours=("wasted_gpu_hours", "sum"),
        avg_gpu_util=("util", "mean")  # Compute average GPU utilization
    )
    .reset_index()
)

# Sort by wasted GPU hours in descending order
top_interactive_wasters = interactive_wasted_gpu.sort_values(
    by="wasted_gpu_hours", ascending=False
)

# Display the top users who wasted the most GPU hours with their avg GPU util
print('Top Users Who Wasted the Most GPU Hours with Interactive/On Demand')
print(top_interactive_wasters.head(10))  # Adjust number as needed


### Find jobs with low util (<5%, basically 0), what users have a high proportion of these? Maybe they are requesting without using
#### NOTE: need to check over logic thoroughly, some incompleteness

In [ ]:
# Step 1: Calculate total GPU hours per job (grouped by job_id)
job_gpu_hours = year_2024_01.groupby('job_id')['gpu_hours'].sum().reset_index(name='total_gpu_hours')
job_avg_util = year_2024_01.groupby('job_id')['util'].mean().reset_index(name='avg_util')

# Merge the GPU hours with the average utilization data
job_data = pd.merge(job_gpu_hours, job_avg_util, on='job_id')

# Step 2: Mask jobs where the average utilization is below 5%
low_util_jobs_by_job = job_data[job_data['avg_util'] < 5]

# Step 3: Merge back with the original dataset to get rows related to those low-util jobs
low_util_jobs = pd.merge(year_2024_01, low_util_jobs_by_job[['job_id']], on='job_id', how='inner')

# Step 4: Calculate total GPU hours per user and total GPU hours for low-util jobs
total_gpu_hours_per_user = year_2024_01.groupby('user')['gpu_hours'].sum().reset_index(name='total_gpu_hours')
low_util_gpu_hours = low_util_jobs.groupby('user')['gpu_hours'].sum().reset_index(name='low_util_gpu_hours')

# Merge counts
user_util_summary = pd.merge(total_gpu_hours_per_user, low_util_gpu_hours, on='user', how='left').fillna(0)

# Step 5: Calculate the percentage of low-util GPU hours per user
user_util_summary['low_util_percent'] = user_util_summary['low_util_gpu_hours'] / user_util_summary['total_gpu_hours']

# Step 6: Filter users with 80% or more low-util GPU hours
consistent_low_util_users = user_util_summary[user_util_summary['low_util_percent'] >= 0.8]

consistent_low_util_users['low_util_gpu_hours'] /= 12

# Visualize the results
plt.figure(figsize=(10, 6))
sns.barplot(
    x='low_util_gpu_hours', 
    y='user', 
    data=consistent_low_util_users.sort_values('low_util_gpu_hours', ascending=False).head(10),
    palette='Blues_r'
)
plt.title('Users with 80% or More GPU Hours Having Low GPU Utilization')
plt.xlabel('Low-Util GPU Hours')
plt.ylabel('User')
plt.grid(axis='x')
plt.show()

# Display the result for reference
print(consistent_low_util_users.sort_values('low_util_gpu_hours', ascending=False))


In [ ]:
year_2024_01[year_2024_01['user']==consistent_low_util_users.sort_values('low_util_gpu_hours', ascending=False)['user'].iloc[0]].options.iloc[0]

### maybe an automated script can reach out to these users each month!